In [1]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import os
import random

from functions.parse_data import synth_dataloader
from multivariate_quantile_regression.network_model import QuantileNetwork

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from cot_train.utils import MLP5

2024-04-22 13:04:59.860938: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 13:04:59.895109: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 13:04:59.895127: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 13:04:59.895963: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 13:04:59.901748: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # CUDA is available, so let's set default device to GPU
    torch.set_default_device(0)
    print("CUDA is available. Using GPU.")
else:
    # CUDA is not available, so let's use the CPU
    print("CUDA is not available. Using CPU.")

# Example usage:
tensor = torch.randn(3, 3)  # Create a tensor on the selected device
print("Tensor is on device:", tensor.device)
device = tensor.device

CUDA is available. Using GPU.
Tensor is on device: cuda:0


In [3]:
#Load data and inspect
df = synth_dataloader('SMHIdata2_newsurf')
df.head(10)

,Cloud_B02,Cloud_B03,Cloud_B04,Cloud_B05,Cloud_B06,Cloud_B07,Cloud_B08,Cloud_B08A,Cloud_B09,Cloud_B10,...,Clear_B11,Clear_B12,Sat_Zenith_Angle,Sun_Zenith_Angle,Azimuth_Diff_Angle,COT,Cloud_Type,Profile_ID,GOT,Water_Vapor
0,0.57804,0.51792,0.54680,0.56000,0.56824,0.57365,0.56583,0.57950,0.37962,0.01949,...,0.04745,0.03758,9.00,68.68,48.59,25.181,3,9543,0.122,0.56
1,0.28975,0.25479,0.29171,0.32868,0.40295,0.44817,0.45809,0.50999,0.17019,0.00067,...,0.75602,0.58746,1.60,73.05,176.23,1.730,2,3672,0.116,0.77
2,0.71170,0.68907,0.73376,0.76922,0.81416,0.83261,0.83383,0.85262,0.63399,0.10050,...,0.65577,0.52408,14.75,42.45,16.45,20.746,4,3564,0.124,0.23
3,0.30316,0.32260,0.39997,0.43564,0.48821,0.52882,0.53207,0.58302,0.22735,0.00072,...,0.87771,0.74346,7.49,55.96,96.60,0.721,1,2993,0.122,0.83
4,0.84968,0.80047,0.83504,0.80686,0.83849,0.86902,0.80830,0.88989,0.26912,0.00051,...,0.81451,0.56093,1.45,51.76,79.44,49.984,3,6226,0.127,4.57
5,0.47968,0.45684,0.48555,0.50933,0.63299,0.71180,0.65449,0.78567,0.27385,0.01380,...,0.91439,0.60915,9.17,64.44,87.17,11.128,2,5251,0.128,4.42
6,0.31475,0.34345,0.35773,0.44913,0.72468,0.78040,0.75561,0.80269,0.34052,0.00156,...,0.90172,0.66072,10.77,68.43,122.12,4.445,1,6703,0.080,0.71
7,0.71760,0.68823,0.71453,0.72291,0.81518,0.87488,0.82043,0.93069,0.37696,0.04403,...,0.94217,0.81028,10.29,41.28,9.38,22.209,5,3031,0.128,2.78
8,0.63913,0.63027,0.67796,0.71480,0.81141,0.86388,0.85365,0.89979,0.65188,0.33643,...,0.89019,0.62145,7.52,48.72,141.93,18.855,4,8434,0.101,0.58
9,0.24895,0.24306,0.24877,0.31625,0.51416,0.57959,0.58756,0.64406,0.27037,0.00287,...,0.75560,0.52477,13.00,67.88,100.70,0.664,5,2395,0.124,0.57


In [4]:
#Choose if to save models and data, if so set paths
save_load=True
if save_load:
    test_name_1 = "COT_est_inp_noCOT"
    main_filepath_1 = 'pytorch_models/'+test_name_1
    test_name_2 = "COT_est_inp_wCOT"
    main_filepath_2 = 'pytorch_models/'+test_name_2
    test_name_3 = "COT_est_inp_wCOT_dum"
    main_filepath_3 = 'pytorch_models/'+test_name_3

Case 1: Exclude COT estimation in input

In [5]:
#Set columns for X and y (input/output features)
X_cols = ['Cloud_B02','Cloud_B03','Cloud_B04','Cloud_B05','Cloud_B06',
          'Cloud_B07','Cloud_B08','Cloud_B08A','Cloud_B09','Cloud_B10','Cloud_B11','Cloud_B12','Sun_Zenith_Angle']
y_cols = ['Clear_B02','Clear_B03','Clear_B04','Clear_B05','Clear_B06',
          'Clear_B07','Clear_B08','Clear_B08A','Clear_B09','Clear_B10','Clear_B11','Clear_B12']

#Find X and y
X=df[X_cols]
y=df[y_cols]

#Separate testdata from rest for 80/10/10 Train/Val/Test split
X_trainval, X_test, y_trainval, y_test=train_test_split(X,y,test_size=0.1,random_state=313)

#Find clear indices in trainval
clear_indices = np.array([])
for i,df_idx in enumerate(X_trainval.index):
    if df['Cloud_Type'][df_idx]==0:
        clear_indices=np.append(clear_indices,i)

#Add noise to X_test, 0 mean with stdev equal to 3% of mean of each feature
np.random.seed(313)
X_test = X_test + np.random.randn(np.shape(X_test)[0],np.shape(X_test)[1]) * np.mean(X.to_numpy(),axis=0)*0.03

In [6]:
#Set up which quantiles to estimate, and find index of estimator (q=0.5)
quantiles=np.array([0.1,0.5,0.9])
est= np.where(quantiles==0.5)[0].item()

#Set up algorithm parameters for both cases
val_size=0.1
num_models=5 #Set number of models in ensemble
batch_size=500
nepochs=1000
lr=0.002
noise_ratio = 0.03
early_break=True
no_nodes = 200
clear_noise = False

In [6]:
#Set up NW
sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols),no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model
    model.fit(X_trainval.to_numpy(),y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break,
            clear_noise=clear_noise,
            clear_indices=clear_indices)
    
    #Save models if wanted
    if save_load:
        filepath=main_filepath_1+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')
    

Epoch 211


Batch number: 100%|██████████| 320/320 [00:01<00:00, 290.76it/s]


Training loss [2.384772] Validation loss [2.3535624]
Epoch 212


Batch number: 100%|██████████| 320/320 [00:01<00:00, 290.98it/s]


Training loss [2.3854601] Validation loss [2.353019]
Epoch 213


Batch number:  25%|██▍       | 79/320 [00:00<00:00, 290.77it/s]


KeyboardInterrupt: 

In [7]:
#Load models
if save_load:
    base_path = main_filepath_1 + '/'
    model_paths = ['model0/model_file','model1/model_file','model2/model_file','model3/model_file','model4/model_file']
    models = [torch.load(base_path+model_paths[i]) for i in range(len(model_paths))]

#Manually set quantiles
quantiles = np.array([0.1,0.5,0.9])
est = np.where(quantiles==0.5)[0].item()

#Initialize dataframe for error metrics and array for ensemble predictions
noCOT_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy(),preds[:,:,est])
    psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds[:,:,est])
    r2=r2_score(y_test.to_numpy(),preds[:,:,est])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
    noCOT_model_metrics=pd.concat([noCOT_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/len(models)

mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,est])
psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds_total[:,:,est])
r2=r2_score(y_test.to_numpy(),preds_total[:,:,est])
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
noCOT_model_metrics=pd.concat([noCOT_model_metrics,tmp_metrics])

#Save metrics if we want to
if save_load:
    noCOT_model_metrics=noCOT_model_metrics.reset_index(drop=True)
    noCOT_model_metrics.to_csv(main_filepath_1+'/model_metrics.csv',index=False)
    

/tmp/ipykernel_54381/3055426930.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  noCOT_model_metrics=pd.concat([noCOT_model_metrics,tmp_metrics])


Case 2: Use Aleksis COT estimation as input to model

In [8]:
#Set up paths for importing COT est models
COT_model_paths = ['smhi_models4/0/model_it_2000000','smhi_models4/1/model_it_2000000','smhi_models4/2/model_it_2000000','smhi_models4/3/model_it_2000000','smhi_models4/4/model_it_2000000']

#Initialize and load COT estimation models
COT_est_models = [MLP5(13, 1, apply_relu=True) for _ in range(len(COT_model_paths))]
for i,model in enumerate(COT_est_models):
    model.load_state_dict(torch.load(COT_model_paths[i],map_location=device))

#Create X for COT estimation (no angles)
X_COTest = X.to_numpy()
#Add noise for fairness
X_COTest = X_COTest + np.random.randn(np.shape(X_COTest)[0],np.shape(X_COTest)[1]) * np.mean(X_COTest,axis=0)*0.03
#Normalize and turn into tensor before input
X_COTest_mu = np.mean(X_COTest,axis=0)
X_COTest_std = np.std(X_COTest,axis=0)
X_COTest_norm = (X_COTest-X_COTest_mu)/X_COTest_std
tX_COTest_norm = torch.Tensor(X_COTest_norm).to(device)
#Make predictions (*50 to denormalize predictions)
COT_preds_total = []
for i,model in enumerate(COT_est_models):
    COT_preds = 50*model(tX_COTest_norm).cpu().detach().numpy()
    #Keep track of ensemble prediction
    if i==0:
        COT_preds_total=COT_preds
    else:
        COT_preds_total=COT_preds_total+COT_preds

COT_preds_total = COT_preds_total/len(COT_est_models)

#Now separate into trainval and test
COT_preds_total_trainval = COT_preds_total[X_trainval.index,0]
COT_preds_total_test = COT_preds_total[X_test.index,0]

#Create new X's including COT estimation
X_trainval_COT = X_trainval.assign(COT_est=COT_preds_total_trainval)
X_test_COT = X_test.assign(COT_est=COT_preds_total_test)

In [7]:
#Create new net with 1 additional input
sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols)+1,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model with X including COT_est
    model.fit(X_trainval_COT.to_numpy(),y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break,
            clear_noise=clear_noise,
            clear_indices=clear_indices)
    
    #Save models if wanted
    if save_load:
        filepath=main_filepath_2+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')
    

Epoch 411


Batch number: 100%|██████████| 320/320 [00:01<00:00, 235.96it/s]


Training loss [2.307456] Validation loss [2.327637]
Epoch 412


Batch number: 100%|██████████| 320/320 [00:01<00:00, 229.81it/s]


Training loss [2.314747] Validation loss [2.3378232]
Epoch 413


...

Training loss [2.311021] Validation loss [2.3337336]
Epoch 419



Batch number: 100%|██████████| 320/320 [00:01<00:00, 294.39it/s]


Training loss [2.3158839] Validation loss [2.3389077]
Epoch 420


Batch number: 100%|██████████| 320/320 [00:01<00:00, 241.39it/s]

Training loss [2.3130164] Validation loss [2.331449]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 320
With validation loss: 2.3136236667633057


In [9]:
#Load models
if save_load:
    base_path = main_filepath_2 + '/'
    model_paths = ['model0/model_file','model1/model_file','model2/model_file','model3/model_file','model4/model_file']
    models = [torch.load(base_path+model_paths[i]) for i in range(len(model_paths))]

#Manually set quantiles
quantiles = np.array([0.1,0.5,0.9])
est = np.where(quantiles==0.5)[0].item()

#Initialize dataframe for error metrics and array for ensemble predictions
wCOT_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test_COT.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy(),preds[:,:,est])
    psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds[:,:,est])
    r2=r2_score(y_test.to_numpy(),preds[:,:,est])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
    wCOT_model_metrics=pd.concat([wCOT_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/num_models

mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,est])
psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds_total[:,:,est])
r2=r2_score(y_test.to_numpy(),preds_total[:,:,est])
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
wCOT_model_metrics=pd.concat([wCOT_model_metrics,tmp_metrics])

#Save metrics if we want to
if save_load:
    wCOT_model_metrics=wCOT_model_metrics.reset_index(drop=True)
    wCOT_model_metrics.to_csv(main_filepath_2+'/model_metrics.csv',index=False)

/tmp/ipykernel_54381/2832525742.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  wCOT_model_metrics=pd.concat([wCOT_model_metrics,tmp_metrics])


Case 3: Categorical dummy COT estimation as input.

In [10]:
#Sort into categories instead
t_is_cloud = 0.025*50 #From Pirinen et. al.
t_thin_cloud = 0.015*50 #From Pirinen et. al.

pred_clear = np.zeros(COT_preds_total.shape)
pred_thin = np.zeros(COT_preds_total.shape)
pred_thick = np.zeros(COT_preds_total.shape)

pred_clear[COT_preds_total<t_thin_cloud]=1
pred_thin[(COT_preds_total>=t_thin_cloud)&(COT_preds_total<t_is_cloud)]=1
pred_thick[COT_preds_total>=t_is_cloud]=1

#Create new Xs including COT dummies
X = X.assign(Clear=pred_clear[:,0])
X = X.assign(Thin=pred_thin[:,0])
X = X.assign(Thick=pred_thick[:,0])

#Now separate into trainval and test
Clear_trainval = pred_clear[X_trainval.index,0]
Clear_test = pred_clear[X_test.index,0]
Thin_trainval = pred_thin[X_trainval.index,0]
Thin_test = pred_thin[X_test.index,0]
Thick_trainval = pred_thick[X_trainval.index,0]
Thick_test = pred_thick[X_test.index,0]

#Create new X's including dummy COT estimation
X_trainval_COT_dum = X_trainval.assign(Clear = Clear_trainval)
X_test_COT_dum = X_test.assign(Clear=Clear_test)
X_trainval_COT_dum = X_trainval_COT_dum.assign(Thin = Thin_trainval)
X_test_COT_dum = X_test_COT_dum.assign(Thin=Thin_test)
X_trainval_COT_dum = X_trainval_COT_dum.assign(Thick = Thick_trainval)
X_test_COT_dum = X_test_COT_dum.assign(Thick=Thick_test)

In [ ]:
#Create new net with 1 additional input
sequence= lambda: nn.Sequential(
    nn.Linear(len(X_cols)+3,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes,no_nodes),
    nn.ReLU(),
    nn.BatchNorm1d(no_nodes),
    nn.Linear(no_nodes, len(quantiles)*len(y_cols)) #Output dimesion is number of quantiles times number of target variables
)

#Initalize models
models = [QuantileNetwork(quantiles=quantiles) for _ in range(num_models)]

#Train models
for i,model in enumerate(models):
    #Find new train/val splits for each model for robustness
    validation_indices=np.array(random.sample(range(len(X_trainval['Cloud_B02'])), int(len(X['Cloud_B02'])*val_size)))
    train_indices=[i for i in range(len(X_trainval['Cloud_B02'])) if np.any(validation_indices==i)==False]  
    #Fit model with X including COT_est
    model.fit(X_trainval_COT_dum.to_numpy(),y_trainval.to_numpy(), 
            train_indices=train_indices, 
            validation_indices=validation_indices, 
            batch_size=batch_size,
            nepochs=nepochs,
            sequence=sequence(),
            lr=lr,
            noise_ratio=noise_ratio,
            early_break=early_break,
            clear_noise=clear_noise,
            clear_indices=clear_indices)
    
    #Save models if wanted
    if save_load:
        filepath=main_filepath_3+'/model'+str(i)
        os.makedirs(filepath,exist_ok=True)
        torch.save(model,filepath+'/model_file')
    

Epoch 331


Batch number: 100%|██████████| 320/320 [00:01<00:00, 307.61it/s]

Training loss [2.3532262] Validation loss [2.3257678]
Epoch 332



Batch number: 100%|██████████| 320/320 [00:01<00:00, 307.55it/s]

Training loss [2.3577774] Validation loss [2.327489]
Epoch 333


...

Training loss [2.3624234] Validation loss [2.3278055]
Epoch 339



Batch number: 100%|██████████| 320/320 [00:01<00:00, 304.76it/s]

Training loss [2.357945] Validation loss [2.3207965]
Epoch 340



Batch number: 100%|██████████| 320/320 [00:01<00:00, 305.34it/s]

Training loss [2.3548305] Validation loss [2.3521652]
---No improvement in 100 epochs, broke early---
Best model out of total max epochs found at epoch 240
With validation loss: 2.3106424808502197


In [11]:
#Load models
if save_load:
    base_path = main_filepath_3 + '/'
    model_paths = ['model0/model_file','model1/model_file','model2/model_file','model3/model_file','model4/model_file']
    models = [torch.load(base_path+model_paths[i]) for i in range(len(model_paths))]

#Manually set quantiles
quantiles = np.array([0.1,0.5,0.9])
est = np.where(quantiles==0.5)[0].item()

#Initialize dataframe for error metrics and array for ensemble predictions
wCOT_dum_model_metrics=pd.DataFrame(columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
preds_total=[]
#Make predictions and evaluate
for i,model in enumerate(models):
    preds = model.predict(X_test_COT_dum.to_numpy())
    #Keep track of ensemble prediction
    if i==0:
        preds_total=preds
    else:
        preds_total=preds_total+preds

    #Find errors
    mse=mean_squared_error(y_test.to_numpy(),preds[:,:,est])
    psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds[:,:,est])
    r2=r2_score(y_test.to_numpy(),preds[:,:,est])
    mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds,quantiles)
    #Add to dataframe
    tmp_metrics=pd.DataFrame(data=[[False,i,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
    wCOT_dum_model_metrics=pd.concat([wCOT_dum_model_metrics,tmp_metrics])


#Now do the same for ensemble predictions
preds_total=preds_total/len(models)

mse=mean_squared_error(y_test.to_numpy(),preds_total[:,:,est])
psnr=QuantileNetwork.PSNR(y_test.to_numpy(),preds_total[:,:,est])
r2=r2_score(y_test.to_numpy(),preds_total[:,:,est])
mean_quantile=QuantileNetwork.mean_marginal_loss(y_test.to_numpy(),preds_total,quantiles)

tmp_metrics=pd.DataFrame(data=[[True,np.nan,mse,psnr,r2,mean_quantile]],columns=['Ensemble_mean','Ensemble_index','MSE','PSNR','R2_score','Mean_Quantile_Loss'])
wCOT_dum_model_metrics=pd.concat([wCOT_dum_model_metrics,tmp_metrics])

#Save metrics if we want to
if save_load:
    wCOT_dum_model_metrics=wCOT_dum_model_metrics.reset_index(drop=True)
    wCOT_dum_model_metrics.to_csv(main_filepath_3+'/model_metrics.csv',index=False)

/tmp/ipykernel_54381/3078255963.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  wCOT_dum_model_metrics=pd.concat([wCOT_dum_model_metrics,tmp_metrics])


Show results:

In [12]:
#Display noCOT results
if save_load:
    file_name = main_filepath_1 + '/model_metrics.csv'
    noCOT_model_metrics=pd.read_csv(file_name)

noCOT_model_metrics

,Ensemble_mean,Ensemble_index,MSE,PSNR,R2_score,Mean_Quantile_Loss
0,False,0.0,0.004911,23.109470,0.838393,0.439172
1,False,1.0,0.004969,23.058931,0.836799,0.440598
2,False,2.0,0.004920,23.101861,0.837970,0.439957
3,False,3.0,0.004950,23.075642,0.837860,0.439557
4,False,4.0,0.004917,23.104641,0.838708,0.440252
5,True,NaN,0.004811,23.199307,0.842014,0.432190


In [13]:
#Display wCOT results
if save_load:
    file_name = main_filepath_2 + '/model_metrics.csv'
    wCOT_model_metrics=pd.read_csv(file_name)

wCOT_model_metrics

,Ensemble_mean,Ensemble_index,MSE,PSNR,R2_score,Mean_Quantile_Loss
0,False,0.0,0.004825,23.186874,0.841511,0.433850
1,False,1.0,0.004875,23.141267,0.840358,0.435206
2,False,2.0,0.004814,23.195924,0.842387,0.433273
3,False,3.0,0.004863,23.152805,0.840461,0.435163
4,False,4.0,0.004870,23.145673,0.840672,0.434619
5,True,NaN,0.004708,23.292763,0.845737,0.425872


In [14]:
#Display wCOT dum results
if save_load:
    file_name = main_filepath_3 + '/model_metrics.csv'
    wCOT_dum_model_metrics=pd.read_csv(file_name)

wCOT_dum_model_metrics

,Ensemble_mean,Ensemble_index,MSE,PSNR,R2_score,Mean_Quantile_Loss
0,False,0.0,0.004975,23.053160,0.836234,0.439600
1,False,1.0,0.004999,23.032947,0.836235,0.440334
2,False,2.0,0.004942,23.082708,0.838040,0.438292
3,False,3.0,0.004932,23.090970,0.837855,0.437804
4,False,4.0,0.004850,23.163883,0.840682,0.435777
5,True,NaN,0.004798,23.211023,0.842523,0.430116
